# **FireWorks Tutorial**

In [1]:
# Importing standard libraries
from fireworks import (Firework, 
                       Workflow, 
                       LaunchPad, 
                       ScriptTask, 
                       TemplateWriterTask, 
                       FileTransferTask, 
                       FiretaskBase, 
                       FWAction)

from fireworks.core.rocket_launcher import rapidfire

# Initialize LaunchPad and Reset
launchpad = LaunchPad()
launchpad.reset("", require_password=False)

2022-05-09 23:08:43,944 INFO Performing db tune-up
2022-05-09 23:08:43,974 INFO LaunchPad was RESET.


## **1.0 Hello World!**

In [2]:
# Create the individual FireWorks
fw1 = Firework(ScriptTask.from_str('echo "Hello World!"'), name="hello")
fw2 = Firework(ScriptTask.from_str('echo "Goodbye!!"'), name="bye")

# Create a workflow that contains the previous Fireworks
wf_hello = Workflow([fw1, fw2], {fw1:fw2}, name="hello world workflow")

# Add workflow to the launchpad
launchpad.add_wf(wf_hello)

# Run the entire workflow
rapidfire(launchpad)

2022-05-09 23:08:43,991 INFO Added a workflow. id_map: {-2: 1, -1: 2}
2022-05-09 23:08:44,188 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-44-188228
2022-05-09 23:08:44,189 INFO Launching Rocket
2022-05-09 23:08:44,204 INFO RUNNING fw_id: 2 in directory: /home/jovyan/launcher_2022-05-09-23-08-44-188228
2022-05-09 23:08:44,213 INFO Task started: ScriptTask.
Hello World!
2022-05-09 23:08:44,231 INFO Task completed: ScriptTask
2022-05-09 23:08:44,254 INFO Rocket finished
2022-05-09 23:08:44,260 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-44-259953
2022-05-09 23:08:44,261 INFO Launching Rocket
2022-05-09 23:08:44,279 INFO RUNNING fw_id: 1 in directory: /home/jovyan/launcher_2022-05-09-23-08-44-259953
2022-05-09 23:08:44,287 INFO Task started: ScriptTask.
Goodbye!!
2022-05-09 23:08:44,299 INFO Task completed: ScriptTask
2022-05-09 23:08:44,319 INFO Rocket finished


## **2.0 Running Multipe FireTasks**

In [3]:
# Create the Fireworks consisting of multiple tasks
ft1 = TemplateWriterTask({'context': {'opt1': 5.0, 'opt2': 'fast method'}, 'template_file': 'simple_template.txt', 'output_file': 'inputs.txt'})
ft2 = ScriptTask.from_str('wc -w < inputs.txt > words.txt')
ft3 = FileTransferTask({'files': [{'src': 'words.txt', 'dest': '~/words.txt'}], 'mode': 'copy'})

# Group all 3 Tasks inside a Firework
fw = Firework([ft1, ft2, ft3], name="Firework with 3 Firetasks")

# Wrap the Firework into a workflow
wf = Workflow([fw], name="Multiple FireTasks Workflow")

# Add workflow to the launchpad
launchpad.add_wf(wf)
rapidfire(launchpad)

2022-05-09 23:08:44,496 INFO Added a workflow. id_map: {-3: 3}
2022-05-09 23:08:44,511 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-44-510929
2022-05-09 23:08:44,512 INFO Launching Rocket
2022-05-09 23:08:44,527 INFO RUNNING fw_id: 3 in directory: /home/jovyan/launcher_2022-05-09-23-08-44-510929
2022-05-09 23:08:44,532 INFO Task started: TemplateWriterTask.
2022-05-09 23:08:44,537 INFO Task completed: TemplateWriterTask
2022-05-09 23:08:44,541 INFO Task started: ScriptTask.
2022-05-09 23:08:44,551 INFO Task completed: ScriptTask
2022-05-09 23:08:44,556 INFO Task started: FileTransferTask.
2022-05-09 23:08:44,558 INFO Task completed: FileTransferTask
2022-05-09 23:08:44,573 INFO Rocket finished


## **3.0 Creating a Custom FireTask**

In [4]:
# Create a custom FireTask that given an array adds them
class AdditionTask(FiretaskBase):
    
    _fw_name = "Addition Task"
    
    def run_task(self, fw_spec):
        input_array = fw_spec['input_array']
        m_sum = sum(input_array)
        
        print(f"The sum of {input_array} is: {m_sum}")
        
        return FWAction(stored_data={'sum': m_sum}, mod_spec=[{'_push': {'input_array': m_sum}}])

In [5]:
# Create the Firework consisting of a our custom AdditionTask
fw_sum = Firework(AdditionTask(), spec={"input_array": [1, 2]}, name="Addition FW")

# Create workflow
wf_sum = Workflow([fw_sum], name="Custom Addition Task Workflow")

# Add to the launchpad and run
launchpad.add_wf(wf_sum)
rapidfire(launchpad)

2022-05-09 23:08:44,837 INFO Added a workflow. id_map: {-4: 4}
2022-05-09 23:08:44,857 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-44-857247
2022-05-09 23:08:44,859 INFO Launching Rocket
2022-05-09 23:08:44,879 INFO RUNNING fw_id: 4 in directory: /home/jovyan/launcher_2022-05-09-23-08-44-857247
2022-05-09 23:08:44,885 INFO Task started: Addition Task.
The sum of [1, 2] is: 3
2022-05-09 23:08:44,887 INFO Task completed: Addition Task
2022-05-09 23:08:44,903 INFO Rocket finished


## **4.0 Creating The Simplest Workflow**

In [6]:
# Create the individual FireWorks
fw1 = Firework(ScriptTask.from_str('echo "To be, or not to be,"'), name="To be, or not to be,")
fw2 = Firework(ScriptTask.from_str('echo "That is the question: "'), name="That is the question:", parents=[fw1])

# Create a workflow that contains the previous Fireworks
wf_hello = Workflow([fw1, fw2], name="Hamlet")

# Add workflow to the launchpad
launchpad.add_wf(wf_hello)

# Run the entire workflow
rapidfire(launchpad)

2022-05-09 23:08:45,072 INFO Added a workflow. id_map: {-6: 5, -5: 6}
2022-05-09 23:08:45,079 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-079343
2022-05-09 23:08:45,080 INFO Launching Rocket
2022-05-09 23:08:45,094 INFO RUNNING fw_id: 6 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-079343
2022-05-09 23:08:45,100 INFO Task started: ScriptTask.
To be, or not to be,
2022-05-09 23:08:45,115 INFO Task completed: ScriptTask
2022-05-09 23:08:45,136 INFO Rocket finished
2022-05-09 23:08:45,141 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-141430
2022-05-09 23:08:45,142 INFO Launching Rocket
2022-05-09 23:08:45,156 INFO RUNNING fw_id: 5 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-141430
2022-05-09 23:08:45,161 INFO Task started: ScriptTask.
That is the question: 
2022-05-09 23:08:45,198 INFO Task completed: ScriptTask
2022-05-09 23:08:45,215 INFO Rocket finished


## **5.0 A Diamond Workflow**

In [7]:
# Create the invidual FireWorks
fw1 = Firework(ScriptTask.from_str('echo "Ingrid is the CEO"'), name="Ingrid is the CEO.")
fw11 = Firework(ScriptTask.from_str('echo "Jack is a manager."'), name="Jack is a manager.", parents=[fw1])
fw12 = Firework(ScriptTask.from_str('echo "Jill is a manager."'), name="Jill is a manager.", parents=[fw1])
fw2 = Firework(ScriptTask.from_str('echo "Kip is an intern."'), name="Kip is an intern.", parents=[fw11, fw12])

# Create a workflow that contains the previous FireWorks
wf_diamond = Workflow([fw1, fw11, fw12, fw2], name="Diamond Workflow")

# Add workflow to the launchpad
launchpad.add_wf(wf_diamond)

# Run the entire workflow
rapidfire(launchpad)

2022-05-09 23:08:45,386 INFO Added a workflow. id_map: {-10: 7, -9: 8, -8: 9, -7: 10}
2022-05-09 23:08:45,396 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-395509
2022-05-09 23:08:45,397 INFO Launching Rocket
2022-05-09 23:08:45,410 INFO RUNNING fw_id: 10 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-395509
2022-05-09 23:08:45,416 INFO Task started: ScriptTask.
Ingrid is the CEO
2022-05-09 23:08:45,430 INFO Task completed: ScriptTask
2022-05-09 23:08:45,451 INFO Rocket finished
2022-05-09 23:08:45,457 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-457098
2022-05-09 23:08:45,458 INFO Launching Rocket
2022-05-09 23:08:45,473 INFO RUNNING fw_id: 8 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-457098
2022-05-09 23:08:45,478 INFO Task started: ScriptTask.
Jill is a manager.
2022-05-09 23:08:45,493 INFO Task completed: ScriptTask
2022-05-09 23:08:45,513 INFO Rocket finished
2022-05-09 23:08:45,518 INFO Created new dir /home/jovyan/launcher

## **6.0 Dynamic Workflow**

In [8]:
# Create the FibonacciAdderTask
class FibonacciAdderTask(FiretaskBase):
    
    _fw_name = "Fibonacci Adder Task"
    
    def run_task(self, fw_spec):
        smaller = fw_spec["smaller"]
        larger = fw_spec["larger"]
        stop_point = fw_spec["stop_point"]
        
        m_sum = smaller + larger
        
        if m_sum < stop_point:
            print(f"The nex Fibonacci number is: {m_sum}")
            # Create a new Fibonacci adder to add to the workflow "dynamically"
            new_fw = Firework(FibonacciAdderTask(), {"smaller": larger, "larger": m_sum, "stop_point": stop_point})
            return FWAction(stored_data={"next_fibnum": m_sum}, additions=[new_fw])
        else:
            print(f"We have now exceeded our limit; (the next Fibonacci number would have been: {m_sum})")
            return FWAction()

In [9]:
# Create a Firework with the new FibonacciAdderTask
fw_fibo = Firework(FibonacciAdderTask(), spec={"smaller": 0, "larger": 1, "stop_point": 100}, name="FibonacciAdder Parent")

# Wrap the Firework as a workflow
wf_fibo = Workflow([fw_fibo], name="Dynamics Workflow")

# Add to the launchpad and run
launchpad.add_wf(wf_fibo)
rapidfire(launchpad)

2022-05-09 23:08:45,861 INFO Added a workflow. id_map: {-11: 11}
2022-05-09 23:08:45,874 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-874310
2022-05-09 23:08:45,875 INFO Launching Rocket
2022-05-09 23:08:45,889 INFO RUNNING fw_id: 11 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-874310
2022-05-09 23:08:45,896 INFO Task started: Fibonacci Adder Task.
The next Fibonacci number is: 1
2022-05-09 23:08:45,897 INFO Task completed: Fibonacci Adder Task
2022-05-09 23:08:45,912 INFO Rocket finished
2022-05-09 23:08:45,917 INFO Created new dir /home/jovyan/launcher_2022-05-09-23-08-45-917289
2022-05-09 23:08:45,918 INFO Launching Rocket
2022-05-09 23:08:45,934 INFO RUNNING fw_id: 12 in directory: /home/jovyan/launcher_2022-05-09-23-08-45-917289
2022-05-09 23:08:45,939 INFO Task started: Fibonacci Adder Task.
The next Fibonacci number is: 2
2022-05-09 23:08:45,940 INFO Task completed: Fibonacci Adder Task
2022-05-09 23:08:45,955 INFO Rocket finished
2022-05-09 23:08:45